# $\S0$ Initial Setup
Perform on every run.

In [ ]:
from google.colab import drive
from IPython.display import clear_output
drive.mount("/content/gdrive")

In [ ]:
PROJ_DIR = "/content/gdrive/MyDrive/L352-Project/Matting"
!mkdir -p {PROJ_DIR}

In [ ]:
!nvidia-smi

# $\S1$ AEMatter

## $\S1.0$ One-off Setup
Clone AEMatter repository and do some minor cleanup.

In [ ]:
# Do only once
!cd {PROJ_DIR} && git clone https://github.com/QLYoo/AEMatter.git
!cd {PROJ_DIR}/AEMatter && cp eval.py eval.py.bak

In [ ]:
# Do only once
%%bash -s $PROJ_DIR
cd "$1/AEMatter" && mkdir -p trimaps images
cd "$1/AEMatter" && cat << EOF > eval.py
import os
import cv2
import numpy as np
import torch
import model
import time

ptrimap='./trimaps/'
pimgs='./images/'
p_outs='./alphas/'

os.makedirs(p_outs,exist_ok=True)

if __name__ == '__main__':
    matmodel = model.AEMatter()
    matmodel.load_state_dict(torch.load('./AEMFIX.ckpt',map_location='cpu')['model'])
    matmodel=matmodel.cuda()
    matmodel.eval()
    with open("lowres-aematter-timings.txt", "a") as f:
        f.write(f"image_name,time_taken\n")
    for idx,file in enumerate(os.listdir(ptrimap)) :
        print(idx)
        start = time.time_ns()
        rawimg=pimgs+file
        trimap=ptrimap+file
        rawimg=cv2.imread(rawimg)
        trimap=cv2.imread(trimap,cv2.IMREAD_GRAYSCALE)
        trimap_nonp=trimap.copy()
        h,w,c=rawimg.shape
        nonph,nonpw,_=rawimg.shape
        newh= (((h-1)//32)+1)*32
        neww= (((w-1)//32)+1)*32
        padh=newh-h
        padh1=int(padh/2)
        padh2=padh-padh1
        padw=neww-w
        padw1=int(padw/2)
        padw2=padw-padw1
        rawimg_pad=cv2.copyMakeBorder(rawimg,padh1,padh2,padw1,padw2,cv2.BORDER_REFLECT)
        trimap_pad=cv2.copyMakeBorder(trimap,padh1,padh2,padw1,padw2,cv2.BORDER_REFLECT)
        h_pad,w_pad,_=rawimg_pad.shape
        tritemp = np.zeros([*trimap_pad.shape, 3], np.float32)
        tritemp[:, :, 0] = (trimap_pad == 0)
        tritemp[:, :, 1] = (trimap_pad == 128)
        tritemp[:, :, 2] = (trimap_pad == 255)
        tritempimgs=np.transpose(tritemp,(2,0,1))
        tritempimgs=tritempimgs[np.newaxis,:,:,:]
        img=np.transpose(rawimg_pad,(2,0,1))[np.newaxis,::-1,:,:]
        img=np.array(img,np.float32)
        img=img/255.
        img=torch.from_numpy(img).cuda()
        tritempimgs=torch.from_numpy(tritempimgs).cuda()
        with torch.no_grad():
            pred=matmodel(img,tritempimgs)
            pred=pred.detach().cpu().numpy()[0]
            pred=pred[:,padh1:padh1+h,padw1:padw1+w]
            preda=pred[0:1,]*255
            preda=np.transpose(preda,(1,2,0))
            preda=preda*(trimap_nonp[:,:,None]==128)+(trimap_nonp[:,:,None]==255)*255
        preda=np.array(preda,np.uint8)
        cv2.imwrite(p_outs+file,preda)
        end = time.time_ns()
        with open("lowres-aematter-timings.txt", "a") as f:
            f.write(f"{file},{end - start}\n")
EOF

Go to the repository website, download the "AEMatter" model, save it as `AEMFIX.ckpt` and upload it into the AEMatter project folder on Google Drive, such that it appears at `./AEMFIX.ckpt` from `$PROJ_DIR`.

## $\S1.1$ Inference
Perform on every run.

In [ ]:
!pip -qq install timm einops

In [ ]:
!cd {PROJ_DIR}/AEMatter && python eval.py